# Zadania – NYC Taxi, Flights oraz Airports


In [0]:
#  Załadowanie danych NYC Taxi do DataFrame
trips_df = spark.table("samples.nyctaxi.trips")

#  Załadowanie danych o opóźnieniach lotów z pliku CSV
flights_df = spark.read.option("header", "true") \
                       .option("inferSchema", "true") \
                       .csv("dbfs:/databricks-datasets/flights/departuredelays.csv")
#  Załadowanie danych o lotniskach z pliku airport-codes-na.txt
airports_df = spark.read.option("header", "true") \
                        .option("inferSchema", "true") \
                        .option("sep", "\t") \
                        .csv("dbfs:/databricks-datasets/flights/airport-codes-na.txt")



---

### Zadanie 1: Obliczanie czasu trwania przejazdu (NYC Taxi)
**Opis:**  
Dodaj do DataFrame `trips_df` nową kolumnę `trip_duration` wyrażoną w minutach, która obliczy różnicę między `tpep_dropoff_datetime` a `tpep_pickup_datetime`. Następnie oblicz średni czas trwania przejazdu dla wszystkich kursów.  
**Podpowiedź:**  
Użyj funkcji takich jak `unix_timestamp()` lub `F.col(...).cast("long")` oraz operacji arytmetycznych.


In [0]:
from pyspark.sql import functions as F
# trips_df
df = trips_df.withColumn(
    'trip_duration',
    (F.unix_timestamp('tpep_dropoff_datetime') - F.unix_timestamp('tpep_pickup_datetime'))).show()


---

### Zadanie 2: Filtrowanie przejazdów weekendowych (NYC Taxi)
**Opis:**  
Wyodrębnij wszystkie przejazdy, które odbyły się w weekendy.  
**Podpowiedź:**  
Skorzystaj z funkcji `dayofweek()` – pamiętaj, że w Spark niedziela może mieć wartość 1 lub 7 (sprawdź dokumentację).



In [0]:
trips_df.filter((F.dayofweek(trips_df.tpep_pickup_datetime) == 1) | (F.dayofweek(trips_df.tpep_pickup_datetime) == 7)).select('tpep_pickup_datetime').show()

---
### Zadanie 3: Obliczanie opłaty za mile (NYC Taxi)
**Opis:**  
Dodaj nową kolumnę `fare_per_mile`, która obliczy koszt przejazdu za jedną milę, poprzez podzielenie opłaty za przejazd (`fare_amount`) przez dystans przejazdu (`trip_distance`). Następnie oblicz średnią wartość tej kolumny dla przejazdów, których `trip_distance` przekracza 2 mile.

**Podpowiedź:**  
Użyj `withColumn()` do stworzenia nowej kolumny oraz `filter()` do ograniczenia danych przed agregacją.


In [0]:
new_df = trips_df.withColumn('fare_per_mile',F.round(F.col('fare_amount')/F.col('trip_distance'),2))
filtered = new_df.filter(F.col('trip_distance')>2).select(F.round(F.avg(F.col('fare_per_mile')),2).alias('AVG')).show()

---

### Zadanie 4: Agregacja według godziny odbioru (NYC Taxi)
**Opis:**  
Wyodrębnij godzinę z `tpep_pickup_datetime` i oblicz średnią opłatę oraz średni dystans przejazdu dla każdej godziny.  
**Podpowiedź:**  
Użyj funkcji `hour()` oraz `groupBy()`.

-


### Zadanie 5: Top 5 kodów pocztowych według łącznej opłaty (NYC Taxi)
**Opis:**  
Dla każdej wartości `pickup_zip` oblicz łączną opłatę (`fare_amount`), a następnie wyświetl 5 kodów pocztowych, dla których suma opłat jest najwyższa.  
**Podpowiedź:**  
Skorzystaj z `groupBy("pickup_zip").agg(F.sum("fare_amount"))` oraz `orderBy()`.


### Zadanie 6: Konwersja i ekstrakcja daty z kolumny `date` (Flights)
**Opis:**  
Przekształć kolumnę `date` w `flights_df` (obecnie typu integer, np. 1011245) na typ datowy, a następnie wyodrębnij miesiąc i dzień do nowych kolumn.  
**Podpowiedź:**  
Możesz użyć funkcji `to_date()`, `substring()` lub `date_format()` – być może najpierw przekształć integer na string.


### Zadanie 7: Średnie opóźnienie dla każdego lotniska wylotu (Flights)
**Opis:**  
Oblicz średnie opóźnienie (`delay`) dla każdego unikalnego lotniska wylotu (`origin`) w `flights_df`, a następnie połącz wynik z `airports_df`, aby wyświetlić nazwę miasta dla lotniska.  
**Podpowiedź:**  
Użyj `groupBy("origin").agg(F.avg("delay"))` oraz operacji `join()` z `airports_df` (po kolumnie `IATA`).

---

### Zadanie 8: Procent lotów na czas dla każdego lotniska (Flights)
**Opis:**  
Dla każdego lotniska wylotu (`origin`), oblicz procent lotów, które wystartowały na czas lub wcześniej (tj. `delay <= 0`).  
**Podpowiedź:**  
Użyj warunkowej agregacji: np. `F.sum(F.when(F.col("delay") <= 0, 1).otherwise(0))` podzielonej przez `count(*)`.

### Zadanie 9: Kategoryzacja opóźnień lotów (Flights)
**Opis:**  
Dodaj do `flights_df` nową kolumnę `status`, która klasyfikuje loty jako: "Early" (delay < 0), "On Time" (delay == 0) oraz "Late" (delay > 0). Następnie policz liczbę lotów w każdej kategorii.  
**Podpowiedź:**  
Użyj funkcji `F.when()` oraz `F.count()` w połączeniu z `groupBy("status")`.


### Zadanie 10: Średni dystans lotów dla każdego lotniska docelowego (Flights)
**Opis:**  
Oblicz średni dystans (`distance`) dla każdego lotniska docelowego (`destination`).  
**Podpowiedź:**  
Skorzystaj z `groupBy("destination").agg(F.avg("distance"))` oraz posortuj wynik malejąco.

### Zadanie 11: Maksymalne opóźnienie na trasie (Flights)
**Opis:**  
Dla każdej pary `origin`–`destination` oblicz maksymalne opóźnienie oraz liczbę lotów na tej trasie.  
**Podpowiedź:**  
Użyj `groupBy("origin", "destination")` i funkcji `F.max("delay")` oraz `F.count("*")`.


### Zadanie 12: Średnie opóźnienie według kombinacji miast (Flights & Airports)
**Opis:**  
Połącz `flights_df` z `airports_df` (dla obu – origin i destination) tak, aby uzyskać informacje o miastach. Następnie oblicz średnie opóźnienie lotów dla każdej kombinacji: miasto wylotu i miasto docelowe.  
**Podpowiedź:**  
Wykonaj dwa JOINy (jeden dla `origin`, drugi dla `destination`) i użyj `groupBy()` na kombinacji `OriginCity` i `DestCity`.


### Zadanie 13: Dummy join między NYC Taxi a Flights
**Opis:**  
Dodaj do `trips_df` oraz `flights_df` nową kolumnę `dummy_date` ustawioną na stałą wartość (np. "2016-02-14"). Następnie wykonaj JOIN obu DataFrame’ów po kolumnie `dummy_date` i sprawdź, ile rekordów uzyskasz.  
**Podpowiedź:**  
Użyj `F.lit("2016-02-14")` w `withColumn()` i wykonaj `join()` na tej kolumnie. Pamiętaj, że wynik to iloczyn rekordów – użyj `limit()` lub `count()` dla przykładu.


### Zadanie 14: Obliczanie czasu trwania przejazdu (NYC Taxi)
**Opis:**  
Dodaj kolumnę `trip_duration` obliczającą czas trwania przejazdu w minutach (różnica między `tpep_dropoff_datetime` a `tpep_pickup_datetime`). Następnie zlicz, ile przejazdów trwało poniżej 5 minut (może to być błąd lub bardzo krótki kurs).  
**Podpowiedź:**  
Użyj funkcji `unix_timestamp()` lub bezpośredniej różnicy między kolumnami typu timestamp oraz `F.datediff()` lub zwykłej arytmetyki.

### Zadanie 15: Statystyczna analiza opóźnień (Flights)
**Opis:**  
Dla każdego lotniska wylotu (`origin`) oblicz odchylenie standardowe opóźnień (`delay`).  
**Podpowiedź:**  
Użyj `groupBy("origin").agg(F.stddev("delay").alias("std_delay"))` oraz posortuj wynik według wartości odchylenia.

---